In [5]:
!pip install optuna scikit-learn


In [ ]:
import optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring='accuracy')

    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best Trial:")
trial = study.best_trial
print(f"Accuracy: {trial.value}")
print("Best hyperparameters:", trial.params)


[I 2025-10-04 17:46:56,134] A new study created in memory with name: no-name-fa47fb92-65e0-4aae-af15-cca333277a21
[I 2025-10-04 17:46:57,227] Trial 0 finished with value: 0.9583333333333334 and parameters: {'n_estimators': 139, 'max_depth': 3, 'min_samples_split': 9, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.9583333333333334.
[I 2025-10-04 17:46:59,522] Trial 1 finished with value: 0.9666666666666668 and parameters: {'n_estimators': 188, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.9666666666666668.
[I 2025-10-04 17:47:00,990] Trial 2 finished with value: 0.9583333333333334 and parameters: {'n_estimators': 173, 'max_depth': 16, 'min_samples_split': 10, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.9666666666666668.
[I 2025-10-04 17:47:02,477] Trial 3 finished with value: 0.9666666666666668 and parameters: {'n_estimators': 195, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 1 with value:

Best Trial:
Accuracy: 0.9666666666666668
Best hyperparameters: {'n_estimators': 188, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 4}


In [6]:
best_params = trial.params
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Akurasi pada data uji: {acc:.4f}")


Akurasi pada data uji: 0.9333


In [7]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_param_importances(study)
